In [1]:
#!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

In [2]:
#!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/parse-faq.ipynb

In [10]:
import minsearch
import json
from dotenv import load_dotenv
from groq import Groq
import os

load_dotenv()

True

In [ ]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

documents=[]

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [8]:
documents[0]

{'text': "dThe purpose of this document is to capture frequently asked technical questions\nThe next cohort starts in Jan 2025. More info at DTC Article.\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start？',
 'course': 'data-engineering-zoomcamp'}

In [9]:
Index = minsearch.Index(
    text_fields = ['question','section','text'],
    keyword_fields = ['course']
)

Index.fit(documents)

In [ ]:
ques = "how to parse document files with python?"

In [ ]:
boost = {'question': 3, 'section' : 0.4}

results = Index.search(
    query=ques,
    filter_dict={'course':'mlops-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [ ]:
results

#### from search script TextSearch Class function

In [ ]:
from Search_uncln import TextSearch

index = TextSearch(
    text_fields = ['question','section','text'],
)
index.fit(documents)

se_results =index.search(
    query = ques,
    n_results= 2, 
    filters= {'course':'mlops-zoomcamp'},
boost = {'question':6, 'section':0.2},
)

In [ ]:
se_results

### chatcompletition, Prompt Template, LLM api call

In [4]:
import minsearch
import json
from dotenv import load_dotenv
from groq import Groq
import os

load_dotenv()

True

In [8]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

documents=[]

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)
        
boost = {'question': 3, 'section' : 0.4}

ques = "how to parse document files with python?" # query

results = Index.search(
    query=ques,
    filter_dict={'course':'mlops-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

Index = minsearch.Index(
    text_fields = ['question','section','text'],
    keyword_fields = ['course']
)

Index.fit(documents)

In [9]:
prompt_template = """ 
You are an expert machine learning and mlops engineering helping a junior engineer as an assitant and guide. 
Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering.
if the CONTEXT does not contain the answer, Output "Not FOUND in the context given" and explain your answer with reasons.

QUESTION: {question}

CONTEXT: {context}
""".strip()

context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

prompt = prompt_template.format(question=ques, context=context).strip()

In [10]:
print(prompt)

You are an expert machine learning and mlops engineering helping a junior engineer as an assitant and guide. 
Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering.
if the CONTEXT does not contain the answer, Output "Not FOUND in the context given" and explain your answer with reasons.

QUESTION: how to parse document files with python?

CONTEXT: section: Module 1: Introduction
question: .gitignore how-to
answer: If you create a folder data and download datasets or raw files in your local repository. Then to push all your code to remote repository without this files or folder please use gitignore file. The simple way to create it do the following steps
1. Create empty .txt file (using text editor or command line)
2. Safe as .gitignore (. must use the dot symbol)
3. Add rules
 *.parquet - to ignore all parquet files
data/ - to ignore all files in folder data

For more pattern read GIT documentation
https://git-scm.com/docs/git

In [11]:

client = Groq(
    api_key=os.getenv("GROQ_API_KEY"),
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="llama3-8b-8192",
)
response.choices[0].message.content

'I understand your question as "How to parse document files with Python?"\n\n Based on the provided context, I did not find a direct answer to your question. However, I found a section related to "Reading large parquet files" which mentions using the Pyspark library. But it\'s not directly relevant to parsing document files with Python.\n\nTherefore, I will output "Not FOUND in the context given" with the reason that the context does not contain a direct answer to parsing document files with Python. If you\'re interested in parsing document files with Python, I can assist you in finding the most relevant and accurate information from other resources or provide some general guidance on how to start.'

In [12]:
def search(query):
    boost = {'question': 3, 'section' : 0.4}

    results = Index.search(
        query=query,
        filter_dict={'course':'mlops-zoomcamp'},
        boost_dict=boost,
        num_results=5
)
    return results

In [13]:
def build_prompt(query, search_results):
    prompt_template = """ 
You are an expert machine learning and mlops engineering helping a junior engineer as an assitant and guide. 
Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering. DO NOT USE OTHER CONTENT OTHER THAN GIVEN CONTEXT!
if the CONTEXT does not contain the answer, Output "Not FOUND in the context given" and explain your answer with reasons.

QUESTION: {question}

CONTEXT: {context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [14]:
def llm_call(prompt):
    client = Groq(
    api_key=os.getenv("GROQ_API_KEY"),
)
    response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="llama3-8b-8192",
)
    return print(response.choices[0].message.content)

In [32]:
Query = "How to use mlflow for experiment tracking?"
def rag(query):
    
    search_results = search(query)
    Prompt = build_prompt(query, search_results)
    answer = llm_call(Prompt)
    return answer

In [33]:
rag(Query)

A question about using MLflow for experiment tracking!

You're asking how to use MLflow for experiment tracking. Unfortunately, I don't see a direct answer to this question in the provided context. The context only provides questions and answers about specific issues with setting up and using MLflow, such as how to delete an experiment permanently, why an experiment is not visible in the MLflow UI, and how to set up MLflow for experiment tracking on GCP.

However, I can provide some general information about using MLflow for experiment tracking. MLflow is an open-source platform for managing the end-to-end machine learning (ML) lifecycle. It allows you to track experiments, version datasets, and reproduce results.

To use MLflow for experiment tracking, you'll need to set up a tracking URI, which is the location where MLflow will store your experiment data. You can set the tracking URI using the `mlflow.set_tracking_uri()` function.

For example, you can use the following code to set u

In [38]:
rag("How to run Kafka?")


Not FOUND in the context given.

The provided context does not contain information on how to run Kafka. It seems to cover topics related to Gitignore, deployment, experiment tracking, and general course questions, but does not mention Kafka.


In [19]:
documents[0]

{'text': "dThe purpose of this document is to capture frequently asked technical questions\nThe next cohort starts in Jan 2025. More info at DTC Article.\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start？',
 'course': 'data-engineering-zoomcamp'}

## With Elastic Search proper Search Engine

In [22]:
from elasticsearch import Elasticsearch

In [23]:
es_client = Elasticsearch("http://localhost:9200")

In [25]:
#es_client.info()   #--to get the setup info

In [ ]:
index_Settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course_faq_questions" #must be lowercase

#create an index, only run one time as it is persistent
es_client.indices.create(index= index_name, body=index_Settings)


In [55]:
from tqdm.auto import tqdm

In [30]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/1026 [00:00<?, ?it/s]

In [60]:
query = "How do I Run Kafka?"

In [61]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index= index_name, body= search_query)     
    
    hits_results = []
    for hit in response['hits']['hits']:
        hits_results.append(hit['_source'])
    
    return hits_results

In [62]:
def rag_with_elastic(query):
    
    search_results = elastic_search(query)
    Prompt = build_prompt(query, search_results)
    answer = llm_call(Prompt)
    return answer

In [63]:
rag_with_elastic(query)

Based on the CONTEXT provided, I can answer the question:

How do I Run Kafka?

The answer is not directly provided in the given CONTEXT. However, we can find related information in the section "Module 6: streaming with kafka" under the question "Java Kafka: How to run producer/consumer/kstreams/etc in terminal".

According to this answer, to run Kafka using Java, you can execute the command:

java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java

In this context, it seems that the command is specific to running a Java producer/consumer/kstreams application. If you're looking for general information on running Kafka, the CONTEXT does not provide a direct answer.

Therefore, my output is:

Not FOUND in the context given

Reason: The CONTEXT does not provide a direct answer to running Kafka, but it does offer related information for running a specific Java application using Kafka.
